In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"C:\Users\Aryaman bhardwaj\Desktop\customer_shopping_behavior.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df.isnull().sum()

In [ ]:
df["Review Rating"] = df.groupby("Category")["Review Rating"].transform(lambda x: x.fillna(x.median()))

In [ ]:
df.isnull().sum()

In [ ]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={ "purchase_amount_(usd)":"purchase_amount"})

In [ ]:
df.columns

In [ ]:
#create a column age_group
labels = ["Young Adults","Adults","Middle-aged","Senior"]
df["age_group"] = pd.qcut(df["age"], q=4, labels = labels)

In [ ]:
df[["age","age_group"]].head(10)

In [ ]:
#create column purchase_frequency_days

frequency_mapping = {
    "Fortnightly" : 14 ,
    "Weekly" : 7,
    "Monthly" : 30,
    "Quarterly" : 90,
    "Bi-Weekly" : 14,
    "Annually" : 365,
    "Every 3 Months" : 90
}

df["purchase_frequency_days"] = df["frequency_of_purchases"].map(frequency_mapping)

In [ ]:
df[["purchase_frequency_days","frequency_of_purchases"]].head(10)

In [ ]:
df[["discount_applied","promo_code_used"]].head(10)

In [ ]:
(df["discount_applied"] == df["promo_code_used"]).all()

In [ ]:
df = df.drop("promo_code_used", axis=1)

In [ ]:
df.columns

In [ ]:
pip install psycopg2-binary sqlalchemy

In [ ]:
from sqlalchemy import create_engine

In [ ]:

# quick test - replace password below
import psycopg2, getpass

password = "MyPass@2025"   # <-- put your postgres password here (or use getpass.getpass())
try:
    conn = psycopg2.connect(host="localhost", port=5432, user="postgres",
                            password=password, dbname="postgres")
    conn.close()
    print("OK: able to connect to postgres on localhost:5432")
except Exception as e:
    print("ERROR:", e)



In [ ]:
# Create database using psycopg2 (run only once)
import psycopg2

password = "MyPass@2025"   # <-- your postgres password
conn = psycopg2.connect(host="localhost", port=5432, user="postgres",
                        password=password, dbname="postgres")
conn.autocommit = True      # important for CREATE DATABASE
cur = conn.cursor()
try:
    cur.execute("CREATE DATABASE customer_behaviour;")
    print("Database customer_behaviour created (or already existed).")
except Exception as e:
    print("Warning / error:", e)
finally:
    cur.close()
    conn.close()



In [ ]:
from sqlalchemy import create_engine

username = "postgres"
password = "MyPass%402025"
host = "localhost"
port = "5432"
database = "customer_behaviour"

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

table_name = "customer"
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data successfully loaded into table '{table_name}' in database '{database}',")